In [2]:
text = """“Houdini” is the lead single of Eminem’s twelfth studio album The Death of Slim Shady (Coup de Grâce). The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese Water Torture Cell, which Eminem replicates in the intro to his 1999 track, “Role Model.”
With the return of long-time collaborator Jeff Bass of the Bass Brothers, Eminem revisits his catchy albeit morbid sound of the early 2000s. This fanfare track sees him callback to his old catchphrases such as Shady’s iconic “Guess who’s back?” line from the 2002 track “Without Me.” The song heavily samples and interpolates the Steve Miller Band’s 1982 track “Abracadabra.”
On October 24, 1926, the legendary Houdini tragically passed away in Detroit, Michigan at the age of 52. Interestingly, Detroit is also the hometown of Eminem, who turns 52 in October 2024.
The single was first teased by Eminem on May 20, 2024, when he posted a teaser to his social media accounts with a text message from “All Contacts” stating, “…and for my last trick!” The text message also has the date of May 31, sparking rumors of Eminem releasing music on that day.
On May 28, 2024, Eminem posted a video to Instagram of him on a FaceTime call with magician David Blaine. At the end of the video, he announced the release of “Houdini” on May 31, then closed out by stating “for my last trick, I’m gonna make my career disappear,” hinting at The Death of Slim Shady (Coup de Grâce) being his final album."""


## AWAN LLM

In [6]:
import requests
import json

def chat_completion(model_name, awanllm_api_key, prompt, preprompt = None):
    url = "https://api.awanllm.com/v1/chat/completions"

    if preprompt is None:
        preprompt = "Your job will be to write the visual of a tiktok video: the images that show on screen. You have to find some nice moments in the story to place pictures that will show on screen. Tell me when you choose an image of what. In this format:\nDURING THE SENTENCE [place the full sentence here] SHOW AN IMAGE OF [place the description of the image].\nFor example:\nDURING THE SENTENCE \"he ate a sandwich\", SHOW AN IMAGE OF a sandwich.\n\nNow it is your turn:\n\nSTORY : "
    else: preprompt = ""
    
    payload = json.dumps({
      "model": model_name,
      "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": preprompt + prompt}  # updated with preprompt and prompt parameters
      ],
      "repetition_penalty": 1.1,
      "temperature": 0.7,
      "top_p": 0.9,
      "top_k": 40,
      "max_tokens": 4096,
      "stream": True
    })
    headers = {
      'Content-Type': 'application/json',
      'Authorization': f"Bearer {awanllm_api_key}"
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response


In [7]:
import os
model_name = "Mistral-7B-Instruct"
api_key = os.environ['API_KEY_AWAN']
# print(api_key)

response = chat_completion("MODEL_NAME", "AWANLLM_API_KEY", "The main character went to the beach and played with her dog.")
print(response.json())
print(response.text)
print(response.content)

{'statusCode': 401, 'message': 'Invalid API Key', 'error': 'Unauthorized'}
{"statusCode":401,"message":"Invalid API Key","error":"Unauthorized"}
b'{"statusCode":401,"message":"Invalid API Key","error":"Unauthorized"}'


## Mistral

In [24]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def get_description_pictures(story):
    api_key = os.environ["MISTRAL_API_KEY"]
    model = "open-mistral-7b"

    client = MistralClient(api_key=api_key)

    preprompt = """Your job will be to write the visual of a tiktok video: the images that show on screen. You have to find some nice moments in the story to place pictures that will show on screen. Tell me when you choose an image of what. In this format:
    DURING THE SENTENCE "[place the full sentence here]", SHOW AN IMAGE OF "[place the description of the image]".
    Answer only with the descriptions of all the images.
    
    For example:
    DURING THE SENTENCE "he ate a sandwich", SHOW AN IMAGE OF "a sandwich".

    Now it is your turn:

    STORY : """

    messages = [
        ChatMessage(role="user", content=preprompt + f'"{story}"'),
    ]

    chat_response = client.chat(
        model=model,
        messages=messages,
    )

    results = chat_response.choices[0].message.content

    return results.split("\n\n")


In [25]:
story = "A group of friends go on a road trip to the beach. They have a fun time swimming, building sandcastles, and enjoying the sun. At night, they roast marshmallows and tell stories."
descriptions = get_description_pictures(story)

print(descriptions)

['DURING THE SENTENCE "A group of friends go on a road trip to the beach", SHOW AN IMAGE OF "A minivan or SUV filled with laughing friends, and the beach in the distance."', 'DURING THE SENTENCE "They have a fun time swimming", SHOW AN IMAGE OF "Friends happily splashing in clear, turquoise water with bright sunshine behind them."', 'DURING THE SENTENCE "Building sandcastles", SHOW AN IMAGE OF "Friends working together to build an elaborate sandcastle on the beach."', 'DURING THE SENTENCE "Enjoying the sun", SHOW AN IMAGE OF "Friends lying on colorful beach towels with sunglasses, soaking up the sun and looking happy."', 'DURING THE SENTENCE "At night, they roast marshmallows", SHOW AN IMAGE OF "Friends sitting around a cozy campfire on the beach, roasting marshmallows on sticks."', 'DURING THE SENTENCE "and tell stories", SHOW AN IMAGE OF "Friends huddled around the campfire, sharing stories and laughing."']


In [26]:
def convert_to_dict(lines):
    visual_dict = {}
    for line in lines:
        print(line)
        if line:
            sentence, image_desc = line.split(', SHOW AN IMAGE OF ', 1)
            sentence = sentence.replace('DURING THE SENTENCE ', '', 1)
            visual_dict[sentence] = image_desc
    return visual_dict

d = convert_to_dict(descriptions)
print(d)

DURING THE SENTENCE "A group of friends go on a road trip to the beach", SHOW AN IMAGE OF "A minivan or SUV filled with laughing friends, and the beach in the distance."
DURING THE SENTENCE "They have a fun time swimming", SHOW AN IMAGE OF "Friends happily splashing in clear, turquoise water with bright sunshine behind them."
DURING THE SENTENCE "Building sandcastles", SHOW AN IMAGE OF "Friends working together to build an elaborate sandcastle on the beach."
DURING THE SENTENCE "Enjoying the sun", SHOW AN IMAGE OF "Friends lying on colorful beach towels with sunglasses, soaking up the sun and looking happy."
DURING THE SENTENCE "At night, they roast marshmallows", SHOW AN IMAGE OF "Friends sitting around a cozy campfire on the beach, roasting marshmallows on sticks."
DURING THE SENTENCE "and tell stories", SHOW AN IMAGE OF "Friends huddled around the campfire, sharing stories and laughing."
{'"A group of friends go on a road trip to the beach"': '"A minivan or SUV filled with laughing 